In [4]:
from sklearn.model_selection import cross_validate
import numpy as np
import os
import sys
import pandas as pd
#import other python scripts for further anlaysis
import reshape
from sklearn.linear_model import RidgeClassifier
#import results
import warnings
warnings.filterwarnings("ignore")
# Initialization of directory information:
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
def classifyCV(sub, task):
    """
    Classifying same subjects (CV) along the same task

    Parameters
    -------------

    Returns
    -------------
    dfCV : DataFrame
        Dataframe consisting of average accuracy across all subjects

    """
    clf=RidgeClassifier()

    taskFC=reshape.permROI(dataDir+task+'/'+sub+'_parcel_corrmat.mat')
    restFC=reshape.permROI(dataDir+'rest/'+sub+'_parcel_corrmat.mat')
    folds=taskFC.shape[0]
    x_train, y_train=reshape.concateFC(taskFC, restFC)
    output = cross_validate(clf, x_train, y_train, cv=folds, return_estimator =True)
    session=0
    i=len(output['estimator'])
    arr=np.empty([i,55278])
    for model in output['estimator']:
        arr[session]=model.coef_
        session=session+1
    fwAve=arr.mean(axis=0)
    indices=reshape.getIndices()
    indices['fw']=fwAve
    lower_triang=indices[['level_0','level_1','variable_0','variable_1','fw']]
    lower_triang.rename(columns={'level_0':'variable_0','level_1':'variable_1','variable_0':'level_0','variable_1':'level_1'},inplace=True)
    full_mat=pd.concat([indices,lower_triang])
    features=full_mat.pivot(index=['level_0','level_1'],columns=['variable_0','variable_1'],values='fw')
    features.sort_index(axis=0,level=1,inplace=True)
    features.sort_index(axis=1,level=1,inplace=True)
    absolute=features.abs()
    dense_mat=absolute.sum(axis=1)
    data={'acc':dense_mat,'roi':roi_sort}
    df=pd.DataFrame(data)
    df.sort_values(by='roi',inplace=True) 
    array=df['acc'].to_numpy()
    array.tofile('/Users/Alexis/Desktop/MSC_Alexis/analysis/output/results/acc/CV/subs/'+task+'/'+train_sub+'.csv', sep = ',')

    


In [5]:
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
taskList=['semantic','motor','mem','glass']
Parcel_params = reshape.loadParcelParams('Gordon333','/Users/Alexis/Desktop/MSC_Alexis/analysis/data/Parcel_info/')
roi_sort = np.squeeze(Parcel_params['roi_sort'])
for task in taskList:
    for train_sub in subList:
        classifyCV(train_sub,task)
        

In [9]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
import numpy as np
import os
import sys
import pandas as pd
#import other python scripts for further anlaysis
import reshape
from sklearn.linear_model import RidgeClassifier
#import results
import warnings
warnings.filterwarnings("ignore")
# Initialization of directory information:
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
splitDict=dict([('MSC01',10),('MSC02',10),('MSC03',8),('MSC04',10),('MSC05',10),('MSC06',9),('MSC07',9),('MSC10',10)])
Parcel_params = reshape.loadParcelParams('Gordon333','/Users/Alexis/Desktop/MSC_Alexis/analysis/data/Parcel_info/')
roi_sort = np.squeeze(Parcel_params['roi_sort'])
def modelAll(train_sub):
    """
    Preparing machine learning model with appropriate data

    Parameters
    -------------
    train_sub : str
            Subject name for training
    test_sub : str
            Subject name for testing

    Returns
    -------------
    total_score : float
            Average accuracy of all folds

    """
    session=splitDict[train_sub]
    split=np.empty((session, 55278))
    count=0
    clf=RidgeClassifier()
    df=pd.DataFrame()
    #train sub
    memFC=reshape.permROI(dataDir+'mem/'+train_sub+'_parcel_corrmat.mat')
    semFC=reshape.permROI(dataDir+'semantic/'+train_sub+'_parcel_corrmat.mat')
    glassFC=reshape.permROI(dataDir+'glass/'+train_sub+'_parcel_corrmat.mat')
    motFC=reshape.permROI(dataDir+'motor/'+train_sub+'_parcel_corrmat.mat')
    restFC=reshape.permROI(dataDir+'rest/corrmats_timesplit/fourths/'+train_sub+'_parcel_corrmat.mat') #keep tasks seperated in order to collect the right amount of days
    restFC=np.reshape(restFC,(10,4,55278)) #reshape to gather correct days
    loo = LeaveOneOut()
    fw=np.empty([memFC.shape[0],55278])
    for train_index, test_index in loo.split(split):
        memtrain=memFC[train_index]
        semtrain=semFC[train_index]
        mottrain=motFC[train_index]
        glatrain=glassFC[train_index]
        Xtrain_task=np.concatenate((memtrain,semtrain,mottrain,glatrain))
        Xtrain_rest=restFC[train_index,:,:]
        Xtrain_rest=np.reshape(Xtrain_rest,(-1,55278))
        ytrain_task = np.ones(Xtrain_task.shape[0], dtype = int)
        ytrain_rest=np.zeros(Xtrain_rest.shape[0], dtype=int)
        X_tr=np.concatenate((Xtrain_task, Xtrain_rest))
        y_tr = np.concatenate((ytrain_task,ytrain_rest))
        clf.fit(X_tr,y_tr)
        features = clf.coef_[0]
        fw[count]=features
        count=count+1
    return fw
"""
    fwAve=fw.mean(axis=0)
    indices=reshape.getIndices()
    indices['fw']=fwAve
    lower_triang=indices[['level_0','level_1','variable_0','variable_1','fw']]
    lower_triang.rename(columns={'level_0':'variable_0','level_1':'variable_1','variable_0':'level_0','variable_1':'level_1'},inplace=True)
    full_mat=pd.concat([indices,lower_triang])
    features=full_mat.pivot(index=['level_0','level_1'],columns=['variable_0','variable_1'],values='fw')
    features.sort_index(axis=0,level=1,inplace=True)
    features.sort_index(axis=1,level=1,inplace=True)
    absolute=features.abs()
    dense_mat=absolute.sum(axis=1)
    data={'acc':dense_mat,'roi':roi_sort}
    df=pd.DataFrame(data)
    df.sort_values(by='roi',inplace=True) 
    array=df['acc'].to_numpy()
    array.tofile('/Users/Alexis/Desktop/MSC_Alexis/analysis/output/results/acc/ALL/subs/'+train_sub+'.csv', sep = ',')

    results=np.empty((333))
    for rowID, null in enumerate(results):
        indices=reshape.getIndices()
        index=indices.index
        condition=indices['level_1']==rowID
        ROI=index[condition]
        ROI_list=ROI.tolist()
        tmp=fwAve[ROI_list]
        row=np.sum(np.abs(tmp))
        results[rowID]=row
    return fw, results
"""
 

"\n    fwAve=fw.mean(axis=0)\n    indices=reshape.getIndices()\n    indices['fw']=fwAve\n    lower_triang=indices[['level_0','level_1','variable_0','variable_1','fw']]\n    lower_triang.rename(columns={'level_0':'variable_0','level_1':'variable_1','variable_0':'level_0','variable_1':'level_1'},inplace=True)\n    full_mat=pd.concat([indices,lower_triang])\n    features=full_mat.pivot(index=['level_0','level_1'],columns=['variable_0','variable_1'],values='fw')\n    features.sort_index(axis=0,level=1,inplace=True)\n    features.sort_index(axis=1,level=1,inplace=True)\n    absolute=features.abs()\n    dense_mat=absolute.sum(axis=1)\n    data={'acc':dense_mat,'roi':roi_sort}\n    df=pd.DataFrame(data)\n    df.sort_values(by='roi',inplace=True) \n    array=df['acc'].to_numpy()\n    array.tofile('/Users/Alexis/Desktop/MSC_Alexis/analysis/output/results/acc/ALL/subs/'+train_sub+'.csv', sep = ',')\n\n    results=np.empty((333))\n    for rowID, null in enumerate(results):\n        indices=reshap

In [2]:
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
for train_sub in subList:
    modelAll(train_sub)

In [36]:
import reshape
import pandas as pd
import numpy as np



In [40]:
df.sort_values(by='roi',inplace=True) 
array=df['acc'].to_numpy()
#df=pd.DataFrame(data)

In [42]:
df

acc  roi
level_0  level_1               
unassign 47       0.119502    0
co       250      0.106775    1
sm       288      0.096657    2
default  48       0.120625    3
         88       0.139094    4
...                    ...  ...
auditory 318      0.109841  328
         319      0.117116  329
default  87       0.108361  330
van      204      0.096460  331
         205      0.111970  332

[333 rows x 2 columns]

In [43]:
mat

level_0   level_1
unassign  0          0.111368
          1          0.090337
          2          0.097816
          3          0.106742
          4          0.094631
                       ...   
pon       328        0.112735
          329        0.094745
          330        0.094830
          331        0.088558
          332        0.119544
Length: 333, dtype: float64

In [12]:
lower_triang=indices[['level_0','level_1','variable_0','variable_1','fw']]
lower_triang.rename(columns={'level_0':'variable_0','level_1':'variable_1','variable_0':'level_0','variable_1':'level_1'},inplace=True)
full_mat=pd.concat([indices,lower_triang])

In [16]:
features=full_mat.pivot(index=['level_0','level_1'],columns=['variable_0','variable_1'],values='fw')


In [21]:
features.sort_index(axis=0,level=1,inplace=True)
features.sort_index(axis=1,level=1,inplace=True)

In [30]:
Parcel_params = reshape.loadParcelParams('Gordon333','/Users/Alexis/Desktop/MSC_Alexis/analysis/data/Parcel_info/')

roi_sort = np.squeeze(Parcel_params['roi_sort'])
roi_sort

array([ 10,  17,  18,  72, 114, 117, 118, 119, 120, 121, 122, 123, 124,
       127, 128, 132, 133, 134, 141, 143, 158, 171, 177, 178, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 290, 291, 295, 296, 299,
       300, 301, 302, 303, 304, 305, 311, 313,   0,   3,   5,  24,  25,
        43,  93, 113, 115, 116, 125, 126, 144, 145, 149, 150, 151, 153,
       155, 156, 161, 164, 183, 185, 199, 219, 224, 256, 258, 277, 278,
       289, 314, 315, 320, 321, 322, 323, 324, 325, 330,   4,   7,  14,
        15,  16,  19,  89,  96,  97,  98, 130, 131, 135, 136, 137, 138,
       139, 140, 165, 168, 174, 175, 176, 250, 254, 255, 257, 262, 263,
       264, 266, 292, 297, 298, 306, 307, 308, 309, 310,   6,   8,  23,
        77,  95, 107, 108, 147, 148, 166, 167, 169, 181, 239, 259, 260,
       271, 272, 275, 276, 318, 319, 326, 327,  40,  41,  42,  48,  50,
        51,  54,  73,  86,  87,  90,  91,  94,  99, 105, 106, 109, 112,
       154, 188, 198, 202, 207, 210, 235, 249, 251, 252, 261, 26

In [63]:
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
Parcel_params = reshape.loadParcelParams('Gordon333','/Users/Alexis/Desktop/MSC_Alexis/analysis/data/Parcel_info/')
roi_sort = np.squeeze(Parcel_params['roi_sort'])
for train_sub in subList:
    fw, results=modelAll(train_sub)
    data={'acc':results,'roi':roi_sort}
    df=pd.DataFrame(data)
    df.sort_values(by='roi',inplace=True) 
    array=df['acc'].to_numpy()#convert back to numpy array for saving to make plots 
    array.tofile('/Users/Alexis/Desktop/MSC_Alexis/analysis/output/results/acc/ALL/subs/'+train_sub+'.csv', sep = ',')

In [11]:
fw=modelAll('MSC06')


In [12]:
import reshape
fwSize=fw.shape[0]
Parcel_params = reshape.loadParcelParams('Gordon333','/Users/Alexis/Desktop/MSC_Alexis/analysis/data/Parcel_info/')
roi_sort = np.squeeze(Parcel_params['roi_sort'])
for i in range(fwSize):
    fold=fw[i]
    indices=reshape.getIndices()
    indices['fw']=fold
    lower_triang=indices[['level_0','level_1','variable_0','variable_1','fw']]
    lower_triang.rename(columns={'level_0':'variable_0','level_1':'variable_1','variable_0':'level_0','variable_1':'level_1'},inplace=True)
    full_mat=pd.concat([indices,lower_triang])
    features=full_mat.pivot(index=['level_0','level_1'],columns=['variable_0','variable_1'],values='fw')
    features.sort_index(axis=0,level=1,inplace=True)
    features.sort_index(axis=1,level=1,inplace=True)
    absolute=features.abs()
    dense_mat=absolute.sum(axis=1)
    data={'acc':dense_mat,'roi':roi_sort}
    df=pd.DataFrame(data)
    df.sort_values(by='roi',inplace=True) 
    array=df['acc'].to_numpy()
    array.tofile('/Users/Alexis/Desktop/MSC_Alexis/analysis/output/results/acc/ALL/foldMSC06/fold'+str(i)+'.csv', sep = ',')

In [35]:
data={'acc':fwAve,'roi':roi_sort}
df=pd.DataFrame(data)
df.sort_values(by='roi',inplace=True) #take the sum of absolute value per row 

ValueError: arrays must all be same length

In [40]:
roi_sort.shape

(333,)

In [1]:
import quest_nullNet as qn
qn.run()

Finished with 14808
Finished with 10824
Finished with 8736
Finished with 4620
Finished with 5264
Finished with 3151
Finished with 494
Finished with 4060
Finished with 2375
Finished with 316
Finished with 564
Finished with 45
Finished with 21


In [6]:
import pandas as pd
import numpy as np
import featSelection as fs
SS_df=pd.DataFrame()
import os
import sys
import reshape
from statistics import mean
#import other python scripts for further anlaysis
# Initialization of directory information:
#thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
outDir = thisDir + 'output/results/subNetwork/'
netRoi=dict([('unassign',14808),('default', 10824),('visual',8736),('fp', 4620),('dan',5264),('van',3151),('salience', 494),('co', 4060),('sm', 2375),('sm-lat', 316),('auditory', 564),('pmn',45),('pon',21)])


for i in netRoi:
    #generate a new index
    idx=netRoi[i]
    SS=fs.modelAll(i)
    SS['feature']=idx
    SS['Network']=i
    SS_df=pd.concat([SS_df,SS])
    print('Finished with '+i)
SS_df.to_csv(outDir+'ALL/acc.csv', index=False)

Finished with unassign
Finished with default
Finished with visual
Finished with fp
Finished with dan
Finished with van
Finished with salience
Finished with co
Finished with sm
Finished with sm-lat
Finished with auditory
Finished with pmn
Finished with pon
